# Notebook 01 — EDA e Limpeza do Dataset **Distribuição de Renda**

**Objetivo:**  
Explorar o dataset bruto (`data/raw/distribuicao-renda.csv`), realizar a análise exploratória de dados (EDA), tratar problemas de qualidade (nulos, outliers, tipos) e gerar um dataset limpo para uso posterior na clusterização.

**Etapas:**
1. Carregamento e inspeção inicial.  
2. Estatísticas descritivas e distribuição das variáveis.  
3. Identificação e tratamento de nulos.  
4. Identificação e tratamento de outliers.  
5. Salvamento de versão limpa em `data/processed/`.  


## 1. Importação de pacotes e configuração de paths


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Configurações globais
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 120)

# Paths
ROOT = Path.cwd().resolve().parents[0] if Path.cwd().name == "notebooks" else Path.cwd()
DATA_RAW = ROOT / "data" / "raw"
DATA_PROCESSED = ROOT / "data" / "processed"
DATA_PROCESSED.mkdir(parents=True, exist_ok=True)

RAW_FILE = DATA_RAW / "distribuicao-renda.csv"
print("Arquivo bruto:", RAW_FILE.exists(), RAW_FILE)


Arquivo bruto: True C:\QuartoSemestre\kmeans-3d-cluster\data\raw\distribuicao-renda.csv


## 2. Carregamento inicial do dataset bruto
- Verificar número de linhas e colunas.  
- Conferir nomes das variáveis.  
- Observar amostra inicial.


In [5]:
# Carregar dataset bruto
df = pd.read_csv(RAW_FILE, sep=";", encoding="utf-8")

# Visualizar shape e primeiras linhas
print("Shape:", df.shape)
df.head()


Shape: (46350, 24)


,Ano-calendário,Ente Federativo,Centil,Quantidade de Contribuintes,Rendimentos Tributaveis - Limite Superior da RTB do Centil [R$ milhões],Rendimentos Tributaveis - Soma da RTB do Centil [R$ milhões],Rendimentos Tributaveis - RTB Acumulada do Centil [R$ milhões],Rendimentos Tributaveis - Média da RTB do Centil [R$],Rendimentos Sujeitos à Tribut. Exclusiva [R$ milhões],Rendimentos Isentos - Lucros e dividendos [R$ milhões],Rendimentos Isentos - Rendim. Sócio/Titular ME/EPP Opt SIMPLES [R$ milhões],Rendimentos Isentos - Outros Rendimentos Isentos [R$ milhões],Despesas Dedutíveis - Previdência [R$ milhões],Despesas Dedutíveis - Dependentes [R$ milhões],Despesas Dedutíveis - Instrução [R$ milhões],Despesas Dedutíveis - Médicas [R$ milhões],Despesas Dedutíveis - Pensão Alimentícia [R$ milhões],Despesas Dedutíveis - Livro-Caixa [R$ milhões],Imposto Devido [R$ milhões],Bens e Direitos - Imóveis [R$ milhões],Bens e Direitos - Móveis [R$ milhões],Bens e Direitos - Financeiros [R$ milhões],Bens e Direitos - Outros Bens e Direitos [R$ milhões],Dívidas e Ônus [R$ milhões]
0,2006,BRASIL,1,241.563,NaN,NaN,NaN,NaN,"235,61","481,27","27,37","1.694,49",NaN,NaN,NaN,NaN,NaN,NaN,"0,16","5.281,59","686,21","6.549,15","1.006,40","1.610,39"
1,2006,BRASIL,2,241.563,NaN,NaN,NaN,NaN,"208,74","483,44","29,36","1.675,01",NaN,NaN,NaN,NaN,NaN,NaN,"0,22","5.295,48","668,82","5.762,77","681,75","694,12"
2,2006,BRASIL,3,241.562,NaN,NaN,NaN,NaN,"219,96","459,87","27,63","1.683,04",NaN,NaN,NaN,NaN,NaN,NaN,"0,31","5.566,27","670,64","5.451,95","377,17","650,98"
3,2006,BRASIL,4,241.563,NaN,NaN,NaN,NaN,"257,01","481,93","31,77","1.699,89",NaN,NaN,NaN,NaN,NaN,NaN,"0,17","5.860,02","678,44","6.104,09","256,16","1.079,20"
4,2006,BRASIL,5,241.562,NaN,NaN,NaN,NaN,"249,88","464,23","28,11","1.697,17",NaN,NaN,NaN,NaN,NaN,NaN,"0,17","5.193,31","682,38","5.592,52","269,28","671,97"


## 3. Checagens iniciais
- Tipos de dados de cada coluna.  
- Presença de valores nulos.  
- Presença de duplicados.  


In [7]:
# Tipos das colunas
print("Tipos de dados:")
print(df.dtypes)

# Contagem de nulos
print("\nValores nulos por coluna:")
print(df.isna().sum())

# Duplicados
print("\nLinhas duplicadas:", df.duplicated().sum())


Tipos de dados:
Ano-calendário                                                                   int64
Ente Federativo                                                                 object
Centil                                                                          object
Quantidade de Contribuintes                                                    float64
Rendimentos Tributaveis - Limite Superior da RTB do Centil [R$ milhões]         object
Rendimentos Tributaveis - Soma da RTB do Centil [R$ milhões]                    object
Rendimentos Tributaveis - RTB Acumulada do Centil [R$ milhões]                  object
Rendimentos Tributaveis - Média da RTB do Centil [R$]                           object
Rendimentos Sujeitos à Tribut. Exclusiva [R$ milhões]                           object
Rendimentos Isentos - Lucros e dividendos [R$ milhões]                          object
Rendimentos Isentos - Rendim. Sócio/Titular ME/EPP Opt SIMPLES [R$ milhões]     object
Rendimentos Isentos - Outro

### 3.1 Conversão de tipos (object → numérico)

Grande parte das variáveis numéricas foi carregada como **texto (object)**, 
pois os valores estão no formato brasileiro (ponto `.` para milhar e vírgula `,` para decimal).

Nesta etapa:
- Identificamos colunas `object` que representam números.  
- Convertemos para `float` (pt-BR → padrão internacional).  
- Ajustamos `Centil` e outras colunas categóricas/numericamente discretas para tipos adequados.  

Esse passo é **fundamental** para que a análise descritiva (célula 4) funcione corretamente.


In [11]:
# 3.1 — Conversão de colunas "object" com números em formato BR para float

import numpy as np
import re

def looks_numeric_br(val: str) -> bool:
    """
    Heurística: detecta strings com dígitos em formato brasileiro:
    - milhares com ponto: 1.234.567
    - decimais com vírgula: 123,45
    - aceita espaços; vazio/None retorna False
    """
    if val is None or (isinstance(val, float) and np.isnan(val)):
        return False
    s = str(val).strip()
    if s == "" or s.lower() in {"nan", "none"}:
        return False
    # Ex.: 1.234,56 | 123,45 | 123456 | 1.234.567 | -1.234,56
    pattern = r"""^\s*[-+]?\d{1,3}(\.\d{3})*(,\d+)?\s*$|^\s*[-+]?\d+(,\d+)?\s*$"""
    return re.match(pattern, s) is not None

def br_to_float(series: pd.Series) -> pd.Series:
    """
    Converte série com números em pt-BR (ponto como milhar, vírgula como decimal) para float.
    Strings vazias, 'NaN', '-' viram NaN.
    """
    s = series.astype(str).str.strip()
    s = s.replace({"": np.nan, "-": np.nan, "NaN": np.nan, "nan": np.nan})
    # remove separador de milhar "." e troca vírgula decimal por "."
    s = s.str.replace(".", "", regex=False).str.replace(",", ".", regex=False)
    return pd.to_numeric(s, errors="coerce")

# 1) Identificar colunas object candidatas (>=60% parecem números BR)
object_cols = [c for c in df.columns if df[c].dtype == "object"]

candidate_numeric_cols = []
for c in object_cols:
    sample = df[c].dropna().astype(str).head(500)  # amostra p/ acelerar
    if len(sample) == 0:
        continue
    frac_numeric_like = sample.apply(looks_numeric_br).mean()
    if frac_numeric_like >= 0.60:
        candidate_numeric_cols.append(c)

print("Candidatas a numéricas (pt-BR):")
for c in candidate_numeric_cols:
    print(" -", c)

# 2) Converter candidatas para float
df_conv = df.copy()
for c in candidate_numeric_cols:
    before_nonnull = df_conv[c].notna().sum()
    df_conv[c] = br_to_float(df_conv[c])
    after_nonnull = df_conv[c].notna().sum()
    print(f"[ok] {c}: não-nulos antes={before_nonnull} | depois={after_nonnull}")

# 3) Ajustar colunas específicas
# Centil deve ser inteiro (pode vir como texto); usa inteiro nulo-capaz (Int64)
if df_conv["Centil"].dtype == "object":
    df_conv["Centil"] = pd.to_numeric(df_conv["Centil"], errors="coerce").astype("Int64")

# Quantidade de Contribuintes já está float64 — se você quiser inteiro:
# (só faça se tiver certeza de que não há frações)
# df_conv["Quantidade de Contribuintes"] = pd.to_numeric(
#     df_conv["Quantidade de Contribuintes"], errors="coerce"
# ).round().astype("Int64")

print("\nTipos após conversão:\n", df_conv.dtypes)

# Substitui df pelo convertido para as próximas células do notebook
df = df_conv

# Visualização rápida
df.dtypes
print(df.head())


Candidatas a numéricas (pt-BR):

Tipos após conversão:
 Ano-calendário                                                                   int64
Ente Federativo                                                                 object
Centil                                                                           int64
Quantidade de Contribuintes                                                    float64
Rendimentos Tributaveis - Limite Superior da RTB do Centil [R$ milhões]        float64
Rendimentos Tributaveis - Soma da RTB do Centil [R$ milhões]                   float64
Rendimentos Tributaveis - RTB Acumulada do Centil [R$ milhões]                 float64
Rendimentos Tributaveis - Média da RTB do Centil [R$]                          float64
Rendimentos Sujeitos à Tribut. Exclusiva [R$ milhões]                          float64
Rendimentos Isentos - Lucros e dividendos [R$ milhões]                         float64
Rendimentos Isentos - Rendim. Sócio/Titular ME/EPP Opt SIMPLES [R$ milhões

## 4. Estatísticas descritivas
- Medidas resumo para todas as variáveis numéricas.  
- Distribuições (histogramas).  
- Relações entre variáveis (correlação).  


## 5. Tratamento de valores nulos
- Estratégia adotada (remoção, imputação, preenchimento com média/mediana).  
- Justificativa da decisão.  


## 6. Tratamento de outliers
- Método escolhido (ex.: IQR, Z-score, winsorização).  
- Justificativa da decisão.  


## 7. Dataset limpo
- Visualizar primeiras linhas após tratamento.  
- Confirmar que não existem mais nulos.  
- Confirmar coerência das distribuições.  


## 8. Salvamento do dataset limpo
Salvar em `data/processed/distribuicao-renda-clean.csv` para uso posterior na clusterização.


## 9. Próximos passos
- Avaliar se a clusterização será feita:
  - Com todos os dados disponíveis,  
  - Com dois anos específicos,  
  - Ou com apenas um ano.  

Essa decisão será baseada na análise criteriosa do dataset limpo.
